###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Unidad de Planificación Rural Agropecuaria - UPRA del Ministerio de Agricultura

In [1]:
import pandas as pd

###Archivo  2cjn-c46k.csv

Contiene  información de la EVA (una investigación que se realiza mediante un mecanismo subjetivo de recolección de información respecto al área, la producción y el rendimiento de la actividad agropecuaria municipal) de base pecuaria para los años 2019 al 2021, referente al inventario de las principales especies pecuarias en los municipios del país.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/2cjn-c46k.csv?$limit=500000", dtype={'c_digo_dane_municipio': str})
df.head()

,departamento,c_digo_dane_departamento,municipio,c_digo_dane_municipio,a_o,especie,total,predios
0,Antioquia,5,Medellín,05001,2019,Caprinos,83,No disponible
1,Antioquia,5,Medellín,05001,2019,Equinos,1003,No disponible
2,Antioquia,5,Medellín,05001,2019,Ovinos,86,No disponible
3,Antioquia,5,Medellín,05001,2019,Bufalos,2,1
4,Antioquia,5,Abejorral,05002,2019,Caprinos,1,No disponible


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11125 entries, 0 to 11124
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   departamento              11125 non-null  object
 1   c_digo_dane_departamento  11125 non-null  int64 
 2   municipio                 11125 non-null  object
 3   c_digo_dane_municipio     11125 non-null  object
 4   a_o                       11125 non-null  int64 
 5   especie                   11125 non-null  object
 6   total                     11125 non-null  int64 
 7   predios                   11125 non-null  object
dtypes: int64(3), object(5)
memory usage: 695.4+ KB


In [5]:
len(df[df['predios']=='No disponible'])

9245

### Eliminación de columnas irrelevantes para el proyecto

In [6]:
relevant_cols = ['c_digo_dane_municipio', 'municipio','departamento', 'a_o', 'especie', 'total']
livestock_count_minagricultura = df[relevant_cols]
livestock_count_minagricultura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11125 entries, 0 to 11124
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   c_digo_dane_municipio  11125 non-null  object
 1   municipio              11125 non-null  object
 2   departamento           11125 non-null  object
 3   a_o                    11125 non-null  int64 
 4   especie                11125 non-null  object
 5   total                  11125 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 521.6+ KB


### Verificar valores nulos

In [7]:
livestock_count_minagricultura.isnull().sum()

,0
c_digo_dane_municipio,0
municipio,0
departamento,0
a_o,0
especie,0
total,0


In [8]:
# anos que abarca la informacion
livestock_count_minagricultura['a_o'].unique()

array([2019, 2020, 2021])

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [9]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['municipio', 'departamento', 'especie']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(livestock_count_minagricultura[column].unique())
    print()


Categorías en la columna 'municipio':
['Medellín' 'Abejorral' 'Abriaquí' ... 'La Pedrera' 'Tarapacá'
 'Mapiripana']

Categorías en la columna 'departamento':
['Antioquia' 'Atlántico' 'Bolívar' 'Boyacá' 'Caldas' 'Caquetá' 'Cauca'
 'Cesar' 'Córdoba' 'Cundinamarca' 'Chocó' 'Huila' 'La Guajira' 'Magdalena'
 'Meta' 'Nariño' 'Norte de Santander' 'Quindío' 'Risaralda' 'Santander'
 'Sucre' 'Tolima' 'Valle del Cauca' 'Arauca' 'Casanare' 'Putumayo'
 'San Andrés y Providencia' 'Amazonas' 'Guainía' 'Guaviare' 'Vaupés'
 'Vichada']

Categorías en la columna 'especie':
['Caprinos' 'Equinos' 'Ovinos' 'Bufalos']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [10]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [11]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    livestock_count_minagricultura.loc[:,col] = livestock_count_minagricultura[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "codigodane" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [12]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigodane" en el df livestock_count_minagricultura

In [14]:
livestock_count_minagricultura.columns

Index(['c_digo_dane_municipio', 'municipio', 'departamento', 'a_o', 'especie',
       'total'],
      dtype='object')

In [15]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
livestock_count_minagricultura.loc[:,'c_digo_dane_municipio'] = livestock_count_minagricultura['c_digo_dane_municipio'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = livestock_count_minagricultura['c_digo_dane_municipio'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [16]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [17]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,c_digo_dane_municipio,count,percentage
0,5,11125,1.0


In [18]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = livestock_count_minagricultura[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['c_digo_dane_municipio']])
    print()

Muestra de registros con longitud 5:
  c_digo_dane_municipio
0                 05001
1                 05001
2                 05001
3                 05001
4                 05002



Nota: Todos los codigos dane de los municipios en el dataset son un string de cinco digitos igual que los de la base de datos, por lo tanto no hay necesidad de hacer ninguna transformacion adicional.

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [19]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [21]:
# Comparar listas de códigos
compare_lists(livestock_count_minagricultura['c_digo_dane_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en livestock_count_minagricultura", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en livestock_count_minagricultura que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [23]:
livestock_count_minagricultura[livestock_count_minagricultura['c_digo_dane_municipio'] == '94663']

,c_digo_dane_municipio,municipio,departamento,a_o,especie,total
11088,94663,MAPIRIPANA,GUAINIA,2021,EQUINOS,33


Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio BarrancoMinas de Guania. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece

In [24]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name
1092,94001,GUAINIA,INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL
1098,94887,GUAINIA,PANA PANA
1099,94888,GUAINIA,MORICHAL


In [25]:
livestock_count_minagricultura.loc[:,'c_digo_dane_municipio'] = livestock_count_minagricultura['c_digo_dane_municipio'].replace( '94663', '94343')

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [26]:
# Comparar listas de códigos
compare_lists(livestock_count_minagricultura['c_digo_dane_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en livestock_count_minagricultura", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en livestock_count_minagricultura que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [28]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['municipio', 'departamento']
livestock_count_minagricultura = livestock_count_minagricultura.drop(columns=columns_to_drop)

In [29]:
# Adicionar columna para trazabilidad de la fuente
livestock_count_minagricultura['source_id'] = 121

In [30]:
livestock_count_minagricultura.columns

Index(['c_digo_dane_municipio', 'a_o', 'especie', 'total', 'source_id'], dtype='object')

In [33]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'c_digo_dane_municipio': 'dane_code',
    'a_o': 'year',
    'especie': 'species',
    'total': 'total',
    'source_id': 'source_id'
}

# Renombrar las columnas
livestock_count_minagricultura.rename(columns=translation_map, inplace=True)

In [34]:
#Estructura final del dataset a integrar a la base de datos
livestock_count_minagricultura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11125 entries, 0 to 11124
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dane_code  11125 non-null  object
 1   year       11125 non-null  int64 
 2   species    11125 non-null  object
 3   total      11125 non-null  int64 
 4   source_id  11125 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 434.7+ KB


## Salvar en archivo csv en el drive

In [35]:
# Guardar en archivos CSV en el drive
livestock_count_minagricultura.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/livestock_count_minagricultura.csv', index=False)